# Data Cleaning and Description
Tyler Christensen, William Lewis, Addison Powell, Jared Smith

## Dataset Descriptions
For our project, we intend to explore a few different branches. Each of these require their own dataset due to the nature of the project, though the methodology will remain largely the same. Our two main branches of exploration are:
- Sentiment Analysis of Casual Texts
    - For this we will use a dataset of over 1.6 million tweets, each labeled with a class related to its sentiment (negative, neutral, or positive).
- Sentiment Analysis of Reviews
    - For this we will use 2 datasets. The first is a set of yelp reviews from <a href="https://www.yelp.com/dataset/documentation/main">this link</a>. There are TODO reviews, each labeled with a score out of 5 stars (TODO: CHECK THIS). The second is a set of 50,000 IMDb Movie reviews, each labeled with either a positive or negative sentiment.

## Validation Set
Before we began to look at the data in full, we sealed off 20% of each set to save for a final analysis. This was split via files, and as such we will not be able to access the validation sets unless we specifically load those files in our code. The split was chosen randomly and immediately saved into a separate file.

TODO: Yelp Dataset is 5GB, and I don't have near enough space on my computer for that. Do it on another computer.

## Data Access
All the data can be found in <a href="https://drive.google.com/drive/folders/1Hp54gH3TQ93ELuzkHJI76C1nXsSI5CpE?usp=sharing">this Google Drive folder.</a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importing Data

In [7]:
# the directory that the data is stored in 
# don't store in git repo, please store it somewhere else on your computer
data_dir = '../../data/'

# load the datasets
twitter_df = pd.read_csv(data_dir + "twitter_data.csv", index_col=0)
imdb_df = pd.read_csv(data_dir + "imdb_data.csv", index_col=0)
# TODO: yelp_df = pd.read_csv(data_dir + "yelp_data.csv", index_col=0)

,target,id,date,flag,user,text
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


## Data Transformation
- TODO: change text data into something models can read

## Feature Analysis
- TODO: Any missing values? How to impute?
- TODO: Variables to drop?
- TODO: What feature engineering is required?

## Data Visualizations and Analysis
- TODO: How is data distributed, is it close to known distribution?
We expect to see a binomial dist heavily weighted to the negative sentiment for a good/bad sentiment analysis (most people on X (formerly known as twitter) are negative).  If we have pairs of either or states such as good/bad sentiment or funny/serious we can make perdictions on a correlation matrix.  Some visualiztions include identifying keywords and creating a heatmap of the intensity they are used within our hidden states.  Other visualiztions could include lists of words or sentence fragements that correspond the most to certain sentiments- TODO: Use 2D, 3D, histograms, and heatmaps to explore correlations

## Conclusions
- TODO: What assumptions do our models rely on, and does our data line up with these models? Or does our dataset change what models would be appropriate?